## Databases ##
1.	Review Database Configurations: note new databases, record any changes to existing, etc.
  * **Database Settings Info.sql**
  **	Included in this script is the sys.databases.log_reuse_wait value for each database.
  * 1. If 0 or 2, this is not an issue and is normal.
  * 2.	If 4, this may indicate an uncommitted transaction is open and needs to be cleaned up. Check uncommitted transactions.sql.
  * 3.	If 6 or 9, this requires some attention to the replication or Availability Groups, respectively.
  * 4.	If other values, investigate accordingly. https://docs.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-databases-transact-sql?view=sql-server-2017
2.	Make sure autogrowth settings are NOT set to 1 MB on any files using Autogrow Change All 1 MB Growth Files.sql
 * We recommend changing the autogrowth settings to a fixed rate between 128 MB and 512 MB depending on the size of the database.
3.	Record any autogrowth events from the past month using Autogrowth Events.sql.
4.	Use Space in files.sql to report any files short on space that are in danger of an autogrowth event.
  * 	We typically report any files with less than 5% free space available, but if the server is checked less frequently than once per month, we want to report anything with less than 10% free space available and also want to manually grow that file to prevent autogrowth events until the next cycle of health checks.
  * Are any log files full? This would indicate FULL recovery + no TLOG backups.
5.	For SharePoint environments, evaluate the correct Recovery model for databases. This cannot be reliably done by database name, so execute the following .ps1 script on a web or app server in the sharepoint farm, or the SQL server if it serves as a single-server farm.
  * Sharepoint Databases.ps1
  * https://technet.microsoft.com/en-us/library/cc678868(v=office.16).aspx
6.	VLF Counts
  * Check the number of VLF’s in all database using VLFs Analysis.sql. Too many VLF’s can be bad for the performance of restore operations and slow recovery operations include the amount of time it takes for a database to come online after a SQL restart. Take a transaction log backup to empty the log file as much as possible, and then shrink the transaction to near-zero using a shrink operation.  Re-grow the log file back to an appropriate size in 8000MB increments. The script handles this.
7.	Ownership
  * Individual employees should not own objects on SQL Server.  In the event their account is LOCKED or DISABLED it could cause problems. Look for this especially on newly created databases.
  ** Database Ownership.sql
  ** Job Owners.sql
8.	Check tempdb data file size – they should all be the same size and have the same autogrowth setting. The initial TempDB data files sizes should be equal to the current TempDB data file sizes.
  * Follow the recommendations in Tempdb Data Files.sql
  *	Pre-grow tempdb data files large so that they never autogrow. If this is a constant issue that cannot be fixed with large initial TempDB sizes (this is rare), consider 1117 – Grow All Files in a FileGroup Equally in SQL versions prior to SQL 2016. 
9.	Age of Statistics –update indexes with outdated statistics (typically by business cycle =  monthly)
  *	Stats Out of Date whileloop.sql to review all databases.
10.	If we have replication, check the Replication Monitor in SSMS. Look for recent errors.
  * Verify that “there are no replicated transactions available” for replication scenarios that are nightly snapshot only.
  *	Verify no error messages for other replication scenarios.
11.	For SQL 2016+, turn on Query Store on any performance-sensitive database. Use default settings. Set to read/write.
  *	select name, is_query_store_on from sys.databases 
12.	Use last known DBCC CHECKDB.sql to verify that a CHECKDB has been run on all databases in at least the last month. If not, we need to examine maintenance plans.

In [ ]:
--Database settings infor.sql
PRINT('Database Settings Output Window')
IF OBJECT_ID('tempdb..#DBSettings') IS NOT NULL
    BEGIN
	   DROP TABLE #DBSettings;
    END;
GO
select 
	name
,	[compatibility_level]	
,	[dbstate] = case when state_desc = 'online' and is_read_only = 1 then state_desc + ' ' +'(Read-Only)' else state_desc end 		
,	recovery_model_desc
,	page_verify_option_desc
,	user_access_desc				--should be MULTI_USER
,	is_auto_close_on				--should be 0
,	is_auto_shrink_on				--should be 0
,	is_auto_create_stats_on			--should be 1 except for some SharePoint db's
,	is_auto_update_stats_on			--should be 1 except for some SharePoint db's
,	is_auto_update_stats_async_on	--should be 1 except for some SharePoint db's
,	log_reuse_wait
,	log_reuse_wait_desc
,	target_recovery_time_in_seconds
,	ProductMajorVersion				= SERVERPROPERTY('ProductMajorVersion')
,	is_trustworthy_on
into #DBSettings
from sys.databases;

--Compatibility Level Check
WITH cteDB (Database_Name, [compatibility_level], State, Up_To_Date)
AS (
SELECT 
 	Database_Name			= name
,	[Compatibility Level]	= [compatibility_level] --should be latest (130 = SQL2016, 120 = SQL2014, 110 = SQL2012, 100 = SQL2008, 90 = SQL2005)
,	[State]					= dbstate		
,	Up_To_Date				= cast(ProductMajorVersion as char(2)) + '0'
from #DBSettings
)
select
	cteDB.*
,	[SQL Server Version]	= SERVERPROPERTY('ProductVersion')
,	[Alter]					= CASE WHEN Up_To_Date is not null THEN 'ALTER DATABASE [' + Database_Name +'] SET COMPATIBILITY_LEVEL = ' + LEFT(convert(varchar(15), SERVERPROPERTY('ProductVersion')),2) + '0;' ELSE NULL END
,	[Revert]				= CASE WHEN Up_To_Date is not null THEN 'ALTER DATABASE [' + Database_Name +'] SET COMPATIBILITY_LEVEL = ' + convert(char(3), [compatibility_level]) + ';' ELSE NULL END
from cteDB
WHERE Up_To_Date <> [compatibility_level]
and state <> 'OFFLINE'
order by [Database_Name];


--Databases where page verify option is not CHECKSUM
--Changing this setting does not instantly put a checksum on every page. Need to do an index REBUILD of all objets to get CHECKSUMS in place, or, it'll happen slowly over time as data is written.
select
 	[Database Name]			= name
,	[Page Verify Option]	= page_verify_option_desc
,	[Message]				= 'Page Verify Option MUST be CHECKSUM!'
,	[Alter]					= 'ALTER DATABASE [' + name +'] SET PAGE_VERIFY CHECKSUM WITH NO_WAIT; --Need to rebuild indexes on all objects in DB to take effect '
,	[Revert]				= 'ALTER DATABASE [' + name +'] SET PAGE_VERIFY ' + page_verify_option_desc COLLATE DATABASE_DEFAULT + ' WITH NO_WAIT;'
,	[State]					= dbstate		
from #DBSettings
where page_verify_option_desc <> 'CHECKSUM'
ORDER BY name;

--Databases where auto-close and/or auto-shrink is enabled. 
--Strongly recommend NEVER enabling either of these two settings.
select 
 	[Database Name]			= name
,	[Is Auto Close On]		= is_auto_close_on		--should be 0
,	[Is Auto Shrink On]		= is_auto_shrink_on		--should be 0
,	[Alter]					= CASE
									WHEN is_auto_close_on = 1 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CLOSE OFF WITH NO_WAIT;'
									WHEN is_auto_shrink_on = 1 THEN 'ALTER DATABASE [' + name + '] SET AUTO_SHRINK OFF WITH NO_WAIT;'
									WHEN is_auto_close_on = 1 AND is_auto_shrink_on = 1 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CLOSE OFF WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_SHRINK OFF WITH NO_WAIT;'
							  ELSE 'N/A'
							  END
,	[Revert]				= CASE
									WHEN is_auto_close_on = 1 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CLOSE ON WITH NO_WAIT;'
									WHEN is_auto_shrink_on = 1 THEN 'ALTER DATABASE [' + name + '] SET AUTO_SHRINK ON WITH NO_WAIT;'
									WHEN is_auto_close_on = 1 AND is_auto_shrink_on = 1 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CLOSE ON WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_SHRINK ON WITH NO_WAIT;'
							  ELSE 'N/A'
							  END
,	[State]					= dbstate	
from #DBSettings
where is_auto_close_on = 1		
   OR is_auto_shrink_on	= 1	
ORDER BY name;

--Databases where auto create and/or auto update stats is disabled
--Recommend enabling these settings.
select 
	[Database Name]					= name
,	[Is Auto Create Stats On]		= is_auto_create_stats_on		--should be 1 except for some SharePoint db's
,	[Is Auto Update Stats On]		= is_auto_update_stats_on		--should be 1 except for some SharePoint db's
,	[Is Auto Update Stats Async On]	= is_auto_update_stats_async_on	--should be 1 except for some SharePoint db's
,	ProductVersion = SERVERPROPERTY('ProductVersion')
,	[Alter]							= CASE
											WHEN is_auto_create_stats_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS ON WITH NO_WAIT;'
											WHEN is_auto_update_stats_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS ON WITH NO_WAIT;'
											WHEN is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC ON WITH NO_WAIT;'
											WHEN is_auto_create_stats_on = 0 AND is_auto_update_stats_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS ON WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS ON WITH NO_WAIT;'
											WHEN is_auto_create_stats_on = 0 AND is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS ON WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC ON WITH NO_WAIT;'
											WHEN is_auto_update_stats_on = 0 AND is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS ON WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC ON WITH NO_WAIT;'
											WHEN is_auto_create_stats_on = 0 AND is_auto_update_stats_on = 0 AND is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS ON WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS ON WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC ON WITH NO_WAIT;'
									  ELSE 'N/A'
									  END
,	[Revert]						= CASE
											WHEN is_auto_create_stats_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS OFF WITH NO_WAIT;'
											WHEN is_auto_update_stats_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS OFF WITH NO_WAIT;'
											WHEN is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC OFF WITH NO_WAIT;'
											WHEN is_auto_create_stats_on = 0 AND is_auto_update_stats_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS OFF WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS OFF WITH NO_WAIT;'
											WHEN is_auto_create_stats_on = 0 AND is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS OFF WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC OFF WITH NO_WAIT;'
											WHEN is_auto_update_stats_on = 0 AND is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS OFF WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC OFF WITH NO_WAIT;'
											WHEN is_auto_create_stats_on = 0 AND is_auto_update_stats_on = 0 AND is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS OFF WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS OFF WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC OFF WITH NO_WAIT;'
									  ELSE 'N/A'
									  END
,	[State]							= dbstate	
from #DBSettings
where is_auto_create_stats_on = 0
   OR is_auto_update_stats_on = 0
   OR (is_auto_update_stats_async_on = 0 and ProductMajorVersion >= 12)
ORDER BY name;

--Databases log reuse wait and description
--Expected types: NOTHING, CHECKPOINT, LOG_BACKUP, ACTIVE_BACKUP_OR_RESTORE, DATABASE_SNAPSHOT_CREATION, AVAILABILITY_REPLICA, OLDEST_PAGE, XTP_CHECKPOINT
--Potentially problematic if long-lasting, research: DATABASE_MIRRORING, REPLICATION, ACTIVE_TRANSACTION, LOG_SCAN, OTHER_TRANSIENT 
select 
	[Database Name]		= name
,	[Log Reuse Wait]	= log_reuse_wait
,	[Description]		= log_reuse_wait_desc
,	[State]				= dbstate		
,	[Recovery Model]	= recovery_model_desc
from #DBSettings
where log_reuse_wait_desc not in ('NOTHING', 'CHECKPOINT', 'LOG_BACKUP', 'ACTIVE_BACKUP_OR_RESTORE', 'DATABASE_SNAPSHOT_CREATION', 'AVAILABILITY_REPLICA', 'OLDEST_PAGE', 'XTP_CHECKPOINT')
ORDER BY name;

--Databases where target recovery time in seconds is < 60 (only applies to 2012+), and recommended in 2016+
--Make sure latest patches are applied first.
select 
	[Database Name]			= name
,	[Target Recovery Time]	= target_recovery_time_in_seconds
,	[Alter]					= 'ALTER DATABASE [' + name + '] SET TARGET_RECOVERY_TIME = 60 SECONDS WITH NO_WAIT'
,	[Revert]				= 'ALTER DATABASE [' + name + '] SET TARGET_RECOVERY_TIME = ' + CAST(target_recovery_time_in_seconds AS VARCHAR(3)) + ' SECONDS WITH NO_WAIT'
,	[State]					= dbstate		
,	ProductMajorVersion
from #DBSettings 
where target_recovery_time_in_seconds = 0
and cast(ProductMajorVersion as int) >= 13
and [name] <> 'master'
ORDER BY name;


--Databases should only have the Trustworthy setting enabled if necessary. The msdb system database is Trustworthy by default. 
select 
	[Database Name]			= name
,	is_trustworthy_on
from #DBSettings
where is_trustworthy_on = 1
and name <> 'msdb'
ORDER BY name;

In [ ]:
--uncommitted transactions.sq
--Returns information on uncommitted transactions
PRINT('Uncommitted Transactions Output Window')
select * from sys.dm_tran_active_transactions tat 
inner join sys.dm_tran_session_transactions tst  on tat.transaction_id = tst.transaction_id

In [ ]:
--Autogrow Change All 1 MB Growth Files.sql
--Run in Results to Text mode
PRINT('Databses with Default Autogrowth Settings in Use')
USE [master]
GO
select 
Alter_Autogrowth_Rates = case when mf.type_desc = 'ROWS' 
	then 'ALTER DATABASE ['+d.name+'] MODIFY FILE ( NAME = N'''+ mf.name+ ''', FILEGROWTH = 256MB );
GO' 
	else 'ALTER DATABASE ['+d.name+'] MODIFY FILE ( NAME = N'''+ mf.name+ ''', FILEGROWTH = 256MB );
GO' 
	end
, mf.*
FROM sys.databases d
inner join sys.master_files mf
on d.database_id = mf.database_id
where (d.state_desc = 'ONLINE')
and (d.is_read_only = 0)
and ((mf.is_percent_growth = 0 and growth = 128) or (mf.is_percent_growth = 1))
/*
ALTER DATABASE [model] MODIFY FILE ( NAME = N'modeldev', FILEGROWTH = 512000KB )
*/

In [ ]:
--Autogrowth Events.sql

--The trace automatically finds _n files, strip off the _nnn. For example, will read all data from log_14.trc, log_15.trc, log_16.trc, log_17.trc, etc. 
--Default trace files are limited to 20mb, and there are at most five of them, so we have 100mb of history. Depends on activity to determine how far back that goes.
PRINT('Autogrowth Events Output Window')

	SELECT 
		DBName				=	g.DatabaseName
	,	DBFileName			=	mf.physical_name
	,	FileType			=	CASE mf.type WHEN 0 THEN 'Row' WHEN 1 THEN 'Log' WHEN 2 THEN 'FILESTREAM' WHEN 4 THEN 'Full-text' END
	,	EventName			=	te.name
	,	EventGrowthMB		=	convert(decimal(19,2),g.IntegerData*8/1024.) -- Number of 8-kilobyte (KB) pages by which the file increased.
	,	EventTime			=	g.StartTime
	,	EventDurationSec	=	convert(decimal(19,2),g.Duration/1000./1000.) -- Length of time (in milliseconds) necessary to extend the file.
	,	CurrentAutoGrowthSet=	CASE
									WHEN mf.is_percent_growth = 1
									THEN CONVERT(char(2), mf.growth) + '%' 
									ELSE CONVERT(varchar(30), convert(decimal(19,2), mf.growth*8./1024.)) + 'MB'
								END
	,	CurrentFileSizeMB	=	convert(decimal(19,2),mf.size*	8./1024.)
	,	d.Recovery_model_Desc
	--,	@tracepath	
	--,	MaxFileSizeMB		=	CASE WHEN mf.max_size = -1 THEN 'Unlimited' ELSE convert(varchar(30), convert(decimal(19,2),mf.max_size*8./1024.)) END
	--select count(1)
	FROM fn_trace_gettable((select substring((select path from sys.traces where is_default =1), 0, charindex('\log_', (select path from sys.traces where is_default =1),0)+4)	+ '.trc'), default) g
	cross apply sys.trace_events te 
	inner join sys.master_files mf
	on mf.database_id = g.DatabaseID
	and g.FileName = mf.name
	inner join sys.databases d
	on d.database_id = g.DatabaseID
	WHERE g.eventclass = te.trace_event_id
	and		te.name in ('Data File Auto Grow','Log File Auto Grow')
	and		g.StartTime > dateadd(d, -7, sysdatetime()) 
	--GROUP BY StartTime,Databaseid, Filename, IntegerData, Duration
	order by StartTime desc;

	SELECT servicename, status_desc, last_startup_time FROM sys.dm_server_services;
GO

In [ ]:
--Space in files.sql
--Observe space in data and log files
--Pregenerated scripts to shrink and/or grow files. Do not shrink unless an unusual/emergency situation has created an overgrown log file. 
--See also "vlfs analysis.sql"
PRINT('Space in Files Output Window')

DECLARE @TempTable TABLE
( DatabaseName varchar(128)
,recovery_model_desc varchar(50)
,DatabaseFileName varchar(500)
,FileLocation varchar(500)
,FileId int
,type_desc varchar(50)
,FileSizeMB decimal(19,2)
,SpaceUsedMB decimal(19,2)
,AvailableMB decimal(19,2)
,FreePercent decimal(19,2)
,shrinkTSQL nvarchar(4000)
,growTSQL nvarchar(4000)
)

--Optional filter for small/unused databases at bottom

INSERT INTO @TempTable
exec sp_MSforeachdb  'use [?]; 
select *
, shrinkTSQL	=	''USE [?];
DBCC SHRINKFILE (N''''''+ DatabaseFileName_______ COLLATE SQL_Latin1_General_CP1_CI_AS +'''''' , 0, TRUNCATEONLY)''
, growTSQL = ''ALTER DATABASE [''+DatabaseName_____________ COLLATE SQL_Latin1_General_CP1_CI_AS+''] 
MODIFY FILE ( NAME = N''''''+DatabaseFileName_______ COLLATE SQL_Latin1_General_CP1_CI_AS +''''''
, '' + CASE WHEN FileSizeMB < 100 THEN ''SIZE = ''+STR(FileSizeMB+64)
			WHEN FileSizeMB < 1000 THEN ''SIZE = ''+STR(FileSizeMB+256)
			WHEN FileSizeMB < 10000 THEN ''SIZE = ''+STR(FileSizeMB+1024)
			WHEN FileSizeMB < 40000 THEN ''SIZE = ''+STR(FileSizeMB+4092)
			ELSE ''SIZE = ''+STR(FileSizeMB+(FileSizeMB*.05)) END +''MB )''
FROM (
SELECT 
  ''DatabaseName_____________'' = d.name
, Recovery			= d.recovery_model_desc
, ''DatabaseFileName_______'' = df.name
, Location			= df.physical_name
, File_ID			= df.File_ID
, df.type_desc
, FileSizeMB		= CAST(size/128.0 as Decimal(9,2))
, SpaceUsedMB		= CAST(CAST(FILEPROPERTY(df.name, ''SpaceUsed'') AS int)/128.0 as Decimal(9,2))
, AvailableMB		= CAST(size/128.0 - CAST(FILEPROPERTY(df.name, ''SpaceUsed'') AS int)/128.0 as Decimal(9,2))
, FreePercent		= CAST((((size/128.0) - (CAST(FILEPROPERTY(df.name, ''SpaceUsed'') AS int)/128.0)) / (size/128.0) ) * 100. as Decimal(9,2))

 FROM sys.database_files df
 CROSS APPLY sys.databases d
 WHERE d.database_id = DB_ID() 
 AND d.is_read_only = 0
 AND df.size > 0) x;
'

SELECT
    *
FROM @TempTable
--where [FreePercent] < 5  and FileSizeMB > 6 --Optional filter for small/unused databases 

ORDER BY FreePercent asc, DatabaseName, FileId

In [ ]:
--VLFs Analysis.sql

--TODO: Rewrite this for SQL 2016 SP2+ and SQL 2017 using sys.dm_db_log_stats instead of DBCC LOGINFO

--If no TSQL scripts generated in the Messages tab, then no log files found with need for VLF maint.

--shows the number of VLF's. CreateLSN=0 for the original created files.
--mf.size /1024, *8 to get MB 
--LogInfo filesize is in bytes, so /1024./1024. to get MB
--Ideally 1 VLF per 500MB. 
--IF log >8 GB, Recreate log in 8000 MB increments.
--A single VLF larger than 4GB could be a problem with backup compression - https://techcommunity.microsoft.com/t5/SQL-Server/Backup-Compression-for-TDE-enabled-Databases-Important-fixes-in/ba-p/385593?advanced=false&collapse_discussion=true&q=MAXTRANSFERSIZE&search_type=thread

--Shrink/regrow step only works for databases with one log file. Why do you have more than one log file anyway? Stop. Think. Ask yourself.

PRINT('VLF Analysis Output Window')
BEGIN TRY
IF EXISTS (select * from tempdb.sys.objects where name like '#Log%')
DROP TABLE #Log
END TRY
BEGIN CATCH
END CATCH

SET NOCOUNT ON

Create Table #Log(
	RecoveryUnitId bigint  null,--SQL 2012 and above only, comment out for <=SQL 2008
    FileID      int not null
  , FileSize_b    bigint not null
  , StartOff	bigint not null
  , FSeqNo      bigint not null
  , [Status]    int not null
  , Parity      bigint not null
  , CreateLSN   decimal(30,0) not null
);  
Exec sp_MSforeachdb N'Use [?]; 
Insert Into #Log  
Exec sp_executesql N''DBCC LogInfo([?]) with no_infomsgs''; 
declare @Co bigint, @Avg_MB decimal(19,2), @LCnt int, @Log_MB decimal(19,2) , @T nvarchar(4000), @Max_MB bigint
select @Log_MB =sum(convert(bigint, mf.size))*8/1024 FROM sys.master_files mf where type=1 and state=0 and db_id()=mf.database_id
select @Co=Count_big(StartOff) ,	@Avg_MB=@Log_MB / Count_big(StartOff), @Max_MB=MAX(FileSize_b/1048576.) from #Log
if (((@Avg_MB <= 64 OR @Avg_MB > 4000) AND @Log_MB > 1024) AND (@Log_MB<8000)) AND EXISTS 
(select 1 FROM sys.databases WHERE is_read_only=0 and state=0 and db_id()=database_id)
BEGIN
		select DBName= db_name(), Co=@Co, Size_MB=@Log_MB, Avg_MB=@Avg_MB
SELECT @T= ''USE [''+d.name+''];
CHECKPOINT
GO
DBCC SHRINKFILE (N''''''+mf.name+'''''' , 0, TRUNCATEONLY)
GO
USE [master]
--Original Size ''+convert(varchar(1000), @Log_MB) +'' MB
ALTER DATABASE [''+d.name+''] MODIFY FILE ( NAME=N''''''+mf.name+'''''', SIZE=''+convert(varchar(30), mf.size*8/1024)+''MB );
GO
''
FROM sys.databases d inner join sys.master_files mf on d.database_id=mf.database_id where type_desc=''log'' and db_name()=d.name
IF @T IS NOT NULL BEGIN
	set @T=@T+''
''
	IF @Co>(@Log_MB/100) and @Co>50
	SELECT DB_NAME()+'' log file too many VLFs.''
	IF (@Avg_MB<64 AND @Log_MB>1024) and @Co>50
	SELECT DB_NAME()+'' log file VLFs too small.''
	IF (@Max_MB >= 4096) 
	SELECT DB_NAME()+'' single log file VLF >4000MB''
	
	print  @T
END	
END
Truncate Table #Log;'


--Had to split this out because of sp_MSforeachdb char limits.
Exec sp_MSforeachdb N'Use [?]
Insert Into #Log  
Exec sp_executesql N''DBCC LogInfo([?]) with no_infomsgs''; 
DECLARE @Co bigint, @Avg_MB decimal(19,2), @LCnt int, @Log_MB decimal(19,2) , @Log_curr bigint, @T nvarchar(4000), @LNeed int, @Ac bigint, @Max_MB bigint
SELECT @Log_MB=sum(convert(bigint, mf.size))*8./1024. FROM sys.master_files mf where type=1 and state=0 and db_id()=mf.database_id
SELECT @Co=Count_big(StartOff),@Avg_MB=@Log_MB/Count_big(StartOff), @Max_MB=MAX(FileSize_b/1048576.) from #Log
IF (@Avg_MB>1024 OR (@Avg_MB<64 AND @Log_MB > 1024)) AND (@Log_MB>8000) AND EXISTS (select 1 FROM sys.databases WHERE is_read_only=0 and state=0 and db_id()=database_id)
BEGIN
SELECT @LCnt=1, @Ac=0
SELECT top 1 @T=''USE [''+d.name+'']
CHECKPOINT
GO
DBCC SHRINKFILE (N''''''+mf.name+'''''' , 0, TRUNCATEONLY)
GO
USE master
GO
--Orig ''+convert(varchar(1000),@Log_MB) +'' MB
''
FROM sys.databases d join sys.master_files mf on d.database_id=mf.database_id where type_desc=''log'' and db_name()=d.name
select @LNeed=@Log_MB/8000
IF (@Log_MB%8000)>=0 
SELECT @LNeed=@LNeed+1 
WHILE (@LCnt<=@LNeed) BEGIN
SET @Log_curr=CASE WHEN @LCnt=1 and @Log_MB<=8000 THEN @Log_MB
WHEN @Log_MB-(8000*@LCnt)>0 THEN 8000 
WHEN @Log_MB-(8000*@LCnt)<0 THEN @Log_MB-(8000*(@LCnt-1))
END				
select @Ac=@Ac+@Log_curr
if @Log_curr>0
select top 1 @T=@T+''ALTER DATABASE [''+d.name+''] MODIFY FILE ( NAME=N''''''+mf.name+'''''', SIZE =''+convert(varchar(1000), @Ac)+'' MB );
GO
''
FROM sys.databases d join sys.master_files mf on d.database_id=mf.database_id where type_desc=''log'' and db_name()=d.name
SELECT @LCnt=@LCnt+1 
END 
END
IF @T IS NOT NULL BEGIN
set @T=@T+''
''
IF @Co>(@Log_MB/100) and @Co>50 SELECT DB_NAME()+'' excessive VLF count.'';
IF @Avg_MB<64 SELECT DB_NAME()+'' VLFs too small'';
IF @Max_MB>=4096 SELECT DB_NAME()+'' single VLF >4GB'';
print @T;
END
Truncate Table #Log;'

Drop Table #Log;


/*
More reference
A VLF is comprised of 1 or more log blocks each of which is an integer multiple of 512 bytes, but no more than 60KB total in size.
----http://www.sqlskills.com/blogs/kimberly/transaction-log-vlfs-too-many-or-too-few/
----http://www.sqlskills.com/blogs/kimberly/8-steps-to-better-transaction-log-throughput/
----https://blogs.msdn.microsoft.com/alwaysonpro/2013/09/27/performing-transaction-log-backups-using-alwayson-availability-group-read-only-secondary-replicas-part-1/
----https://www.red-gate.com/simple-talk/sql/database-administration/sql-server-transaction-log-fragmentation-a-primer/
---"If you need a 2GB log then just create that as one step. 
---If you need a 20GB log, create that as 8GB, then extend it to 16GB and then to 20GB"
--Optimal size for Avg_MB for VLF's is 500MB.

--displays each transaction log size and space used. 
--Dbcc sqlperf (logspace)  --replaced, look for "space in log files.sql"


*/

/*

--Script to test database, create suboptimal VLF's
USE [w]
GO
DBCC SHRINKFILE (N'w2016_log' , 0, TRUNCATEONLY)
GO

USE [master]
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1001MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1002MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1003MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1004MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1005MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1006MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1007MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1008MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1009MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1010MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1011MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1012MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1013MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1014MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1015MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1016MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1017MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1018MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1019MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1020MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1021MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1022MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1023MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1024MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1025MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1026MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1027MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1028MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1029MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1030MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1031MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1032MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1033MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1034MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1035MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1036MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1037MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1038MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1039MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1040MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1041MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1042MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1043MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1044MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1045MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1046MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1047MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1048MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1049MB );
GO
--Won't show up in query above until here
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1050MB );
GO
--further testing the 8 GB growth pattern
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=8000MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=8001MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=16001MB );
GO
*/

/*

--Sample usage

USE [Tfs_Warehouse]
DBCC SHRINKFILE (N'Tfs_Warehouse_log' , 0, TRUNCATEONLY);
GO
USE [master]
--Original Size 1024.00 MB
ALTER DATABASE [Tfs_Warehouse] MODIFY FILE ( NAME=N'Tfs_Warehouse_log', SIZE=1024MB );
GO



USE [Tfs_SparkyBox]
DBCC SHRINKFILE (N'Tfs_SparkyBox_log' , 0, TRUNCATEONLY);
GO
USE [master]
--Original Size 356.00 MB
ALTER DATABASE [Tfs_SparkyBox] MODIFY FILE ( NAME=N'Tfs_SparkyBox_log', SIZE=356MB );
GO
*/


In [ ]:
--Database Ownership.sql
--Find database owners that are not desired
PRINT('Database Ownership Output Window')

declare @Desired_DB_owner varchar(255) = 'sa' --'sa' is just an example, change to desired service account, example: domain\accountname

select 
	database_name = d.name
,	principal_name = SUSER_SNAME (d.owner_sid)
,	set_to_desired = 'alter authorization on database::[' + d.name + '] to [' + @Desired_DB_owner + ']' 
,	set_to_current =  case when SUSER_SNAME (d.owner_sid) <> @Desired_DB_owner THEN 'alter authorization on database::[' + d.name + '] to [' + SUSER_SNAME (d.owner_sid) + ']' ELSE NULL END
,	* 
from sys.databases d
where SUSER_SNAME (d.owner_sid) <> @Desired_DB_owner

In [ ]:
--Job Owners.sql
PRINT('Job Owner Output Window')
use msdb
go
--TODO Change @owner_login_name to desired SQL agent service account to own the job

declare @Desired_job_owner varchar(255) = 'SPARKHOUND\svcaccount' --'sa' is just an example, change to desired service account, example: domain\accountname

--sql 2005 and above
select owner = SUSER_SNAME (j.owner_sid), jobname = j.name, j.job_id
,	change_tsql = N'EXEC msdb.dbo.sp_update_job @job_id=N'''+convert(nvarchar(100), j.job_id)+N''', @owner_login_name=N'''+@Desired_job_owner+''''
,	revert_tsql = N'EXEC msdb.dbo.sp_update_job @job_id=N'''+convert(nvarchar(100), j.job_id)+N''', @owner_login_name=N'''+SUSER_SNAME (j.owner_sid)+''''
from sysjobs j
left outer join sys.server_principals  sp on j.owner_sid = sp.sid
where 
	(sp.name not in ('sa','distributor_admin','NT SERVICE\ReportServer') 
	 and sp.name <> @Desired_job_owner
	 and sp.name not like '##%')
	or sp.name is null 

/*
--sql 2000
select sp.name, j.name, j.job_id from msdb.dbo.sysjobs j
left outer join master.dbo.syslogins sp on j.owner_sid = sp.sid
where sp.name not in ('sa','distributor_admin') or sp.name is null
--EXEC msdb.dbo.sp_update_job @job_id=N'8eab379e-958e-4576-92ae-b5999aeec01c', @owner_login_name=N'distributor_admin'
*/

/*
--Sample usage

EXEC msdb.dbo.sp_update_job @job_id=N'BDAFAC9B-1705-4E47-9C26-6C4B813CB165', @owner_login_name=N'sa'
EXEC msdb.dbo.sp_update_job @job_id=N'987AF666-A516-4847-8BA3-73DE337CFF94', @owner_login_name=N'sa'
EXEC msdb.dbo.sp_update_job @job_id=N'AEBF4C5C-EC6D-4635-96B6-797BF4AEEC62', @owner_login_name=N'sa'

*/

In [ ]:
--Tempdb Data Files.sql
--TempDB data files should all be the same size, same autogrowth settings
--Only displays TempDB data files.
PRINT('TempDB Data Files Output Window')
USE tempdb
GO
DECLARE @cpu_count int;
SELECT @cpu_count = cpu_count from sys.dm_os_sys_info;

--data files
select mf.name
, CurrentSize_MB = (d.size*8.)/1024. --actual current file size
, InitialSize_MB = (mf.size*8.)/1024. --initial file size
, GrowthMb = (mf.growth*8.)/1024.
, mf.is_percent_growth
, MaxFileSizeMB = CASE WHEN mf.max_size > -1 THEN cast((mf.max_size*8.)/1024. as varchar(100)) ELSE 'Unlimited' END -- "-1" is unlimited
, Recommendation = CASE WHEN d.size > mf.size THEN 'Increase TempDB Data files size to match or exceed current size.' + CHAR(10) ELSE '' END +
CASE WHEN mf.size <> AVG(mf.size) OVER (PARTITION BY mf.database_id) THEN 'Set all TempDB Data files to the same initial size.' + CHAR(10) ELSE '' END + 
CASE WHEN d.size <> AVG(d.size) OVER (PARTITION BY mf.database_id) THEN 'Grow TempDB Data files to the same current size.' + CHAR(10) ELSE '' END +
CASE WHEN mf.growth <> AVG(mf.growth) OVER (PARTITION BY mf.database_id) THEN 'Match all TempDB Data files autogrowth rates.' + CHAR(10) ELSE '' END +
CASE WHEN mf.max_size <> AVG(mf.max_size) OVER (PARTITION BY mf.database_id) THEN 'Match all TempDB Data files max file size.' + CHAR(10) ELSE '' END +
CASE WHEN count(mf.file_id) OVER (PARTITION BY mf.database_id) > @cpu_count THEN 'Too many TempDB Data files, reduce to '+cast(@cpu_count as varchar(3)) + ' or lower.' + CHAR(10) ELSE '' END
, mf.physical_name
, volume_letter = UPPER(vs.volume_mount_point)
, file_system_type
, drive_size_GB = (CONVERT(decimal(19,2), vs.total_bytes/1024./1024./1024. ))
, drive_free_space_GB = (CONVERT(decimal(19,2), vs.available_bytes/1024./1024./1024. ))
, drive_pct_free = (CONVERT(DECIMAL(5,2), vs.available_bytes * 100.0 / vs.total_bytes))
from sys.master_files mf
inner join tempdb.sys.database_files d
on mf.file_id = d.file_id
and mf.database_id = db_id()
cross apply sys.dm_os_volume_stats(mf.database_id, mf.file_id) vs --only return volumes where there is database file (data or log)
where d.type_desc = 'rows'
order by mf.file_id asc

--log file
select mf.name
, CurrentSize_MB = (d.size*8.)/1024. --actual current file size
, InitialSize_MB = (mf.size*8.)/1024. --initial file size
, GrowthMb = (mf.growth*8.)/1024.
, mf.is_percent_growth
, MaxFileSizeMB = CASE WHEN mf.max_size > -1 THEN cast((mf.max_size*8.)/1024. as varchar(100)) ELSE 'Unlimited' END -- "-1" is unlimited
, Recommendation = CASE WHEN d.size > mf.size THEN 'Increase TempDB Log file size to match or exceed current size.' + CHAR(10) ELSE '' END
, mf.physical_name
, volume_letter = UPPER(vs.volume_mount_point)
, file_system_type
, drive_size_GB = (CONVERT(decimal(19,2), vs.total_bytes/1024./1024./1024. ))
, drive_free_space_GB = (CONVERT(decimal(19,2), vs.available_bytes/1024./1024./1024. ))
, drive_pct_free = (CONVERT(DECIMAL(5,2), vs.available_bytes * 100.0 / vs.total_bytes))
from sys.master_files mf
inner join tempdb.sys.database_files d
on mf.file_id = d.file_id
and mf.database_id = db_id() 
cross apply sys.dm_os_volume_stats(mf.database_id, mf.file_id) vs --only return volumes where there is database file (data or log)
where d.type_desc = 'log'
order by mf.file_id asc;

SELECT servicename, status_desc, last_startup_time FROM sys.dm_server_services;
GO
--To resize tempdb: 

/*
USE [master]
GO
ALTER DATABASE [tempdb]   MODIFY FILE ( NAME = N'<tempdfilename>'  , SIZE = <Desired File Size>);
GO

USE [tempdb]
GO
CHECKPOINT
GO
DBCC SHRINKFILE (N'<tempdfilename>' , 0, TRUNCATEONLY);
GO
*/

In [ ]:
--Stats Out of Date whileloop.sq
--This script only works in SQL2008R2SP2+ or SQL2012SP1+. 

--This generates an UPDATE STATISTICS script for all databases. Can be used in a maintenance plan (see bottom). Safe to execute. 
--This version does not work in Azure SQL DB. Instead, see toolbox\stats out of date.sql.
--Use toolbox\stats out of date.sql to examine a particular database.

--TODO BEFORE EXECUTING: comment out three lines below in <SQL2014 because incremental stastics not supported.
PRINT('Stats out of Date Output Window')
declare @tsql nvarchar(max) = 
 N'use [?];
    SELECT distinct
--			s.name AS SchemaName 
--          , o.name AS ObjectName 
--          , STA.name AS StatName 
--		  , Object_Type = ISNULL(i.type_desc + '' Index'', ''Statistics Object'')
--          , Stats_Last_Updated = ISNULL(sp.last_updated, o.create_date)
--		  , Rows_Changed = ISNULL(sp.modification_counter,0) --Rows Changed since last update
			--Below block only works in SQL 2014+, comment out this line in prior versions.
			   /*
		  , PartitionNumber = CASE WHEN MAX(p.partition_number) OVER (PARTITION by STA.name, i.name)  > 1 THEN p.partition_number ELSE null END
		  , STA.is_incremental --Only works in SQL 2014+, comment out this line in prior versions.
		  */
--		  , ''use [?]'' AS [?],
		   TSQL = CASE WHEN i.type_desc like ''%columnstore%'' THEN NULL ELSE
		    N''USE [?]; '' + 
			N''UPDATE STATISTICS '' 
               + QUOTENAME(s.name) + N''.'' + QUOTENAME(o.name) + N'' '' 
               + QUOTENAME(STA.name) + N'' '' 
			   + ''WITH RESAMPLE''
			   --Below block only works in SQL 2014+, comment out this line in prior versions.
			   /*
			   + CASE WHEN 
						STA.Is_Incremental = 1 and  --Only works in SQL 2014+, comment out this line in prior versions.
						MAX(p.partition_number) OVER (PARTITION by STA.name, i.name)  > 1 THEN '' ON PARTITIONS ('' + cast(p.partition_number as varchar(5)) + '') '' ELSE '''' END
               */
			END
   FROM sys.objects  o   
		 INNER JOIN sys.stats STA ON STA.object_id = o.object_id  
			CROSS APPLY sys.dm_db_stats_properties (STA.object_id, STA.stats_id) sp -- Only works in SQL2008R2SP2+ or SQL2012SP1+
         INNER JOIN sys.schemas AS s 
             ON o.schema_id = s.schema_id 
		 LEFT OUTER JOIN sys.indexes as i
			on i.index_id = STA.stats_id
			and (i.type_desc not like ''%columnstore%'')
			--Below block only works in SQL 2014+, comment out this line in prior versions.
			 /*
	     LEFT OUTER join sys.dm_db_partition_stats p 
			on (
			STA.Is_Incremental = 1 and  --Only works in SQL 2014+, comment out this line in prior versions. 
			p.object_id = o.object_id  and 
			i.index_id = p.index_id
			)
			*/
         LEFT JOIN 
         (SELECT IUS.object_id 
                ,MIN(ISNULL(IUS.last_user_update, IUS.last_system_update)) AS LastUpdate 
          FROM sys.dm_db_index_usage_stats AS IUS 
          WHERE database_id = DB_ID() 
                AND NOT ISNULL(IUS.last_user_update, IUS.last_system_update) IS NULL 
          GROUP BY IUS.object_id 
         ) AS IUS 
             ON IUS.object_id = STA.object_id 
    WHERE o.type IN (''U'', ''V'')    -- only user tables and views 
          AND DATEDIFF(d, ISNULL(STATS_DATE(STA.object_id, STA.stats_id), N''1900-01-01'')  , IUS.LastUpdate) > 30 --indexes that haven''t been updated in the last month
		  AND sp.modification_counter > 10000
    OPTION (MAXDOP 1);
	print ''[?]'' '

declare @dblist table (id int not null identity(1,1) primary key, dbname sysname not null)
declare @tsqllist table (id int not null identity(1,1) primary key, tsqltext nvarchar(4000) not null) 
declare @x int = 1, @xmax int = null, @dbname sysname, @runtsql nvarchar(max) = null
insert into @dblist 
select name from sys.databases
where state_desc = 'online' and (database_id > 4 or name = 'msdb')
select @xmax = max(id) from @dblist l

while (@x <= @xmax)
BEGIN
	select @dbname = dbname from @dblist l where @x = id

	select @runtsql = replace(@tsql, N'?', @dbname)

	--generates scripts, does not actually perform the UPDATE STATISTICS. See below.
	--Writes all the TSQL into a table variable which is displayed later
	insert into @tsqllist (tsqltext) 
	exec sp_executesql @runtsql --safe, does not actually update stats
	
	set @x = @x + 1
END

--Shows all stats in all databases that need to be updated
select * from @tsqllist

--OPTIONALLY - execute all UPDATE Stats
/*
declare @s int = 1, @scount int = null
select @scount = max(id), @runtsql = null from @tsqllist l
while (@s <= @scount)
BEGIN
	
	--actually executes the scripts.
	select @runtsql = tsqltext from @tsqllist where id = @s
	exec sp_executesql @runtsql
	
	set @s = @s + 1
	
END
*/




/*

USE [WideWorldImporters]; UPDATE STATISTICS [Sales].[InvoiceLines] [_WA_Sys_0000000D_1E6F845E] WITH RESAMPLE
USE [WideWorldImporters]; UPDATE STATISTICS [Sales].[Invoices] [_WA_Sys_0000000A_7849DB76] WITH RESAMPLE

*/

In [ ]:
--last known DBCC CHECKDB.sql
PRINT('Last Known Good DBCC CheckDB')
EXEC sp_MSforeachdb '
--Table variable to capture the DBCC DBINFO output, look for the field we want in each database output
DECLARE @DBCC_DBINFO TABLE (ParentObject VARCHAR(255) NOT NULL, [Object] VARCHAR(255)  NOT NULL, [Field] VARCHAR(255) NOT NULL 
INDEX idx_dbinfo_field CLUSTERED --just this line is SQL 2014+ only
, [Value] VARCHAR(255));
INSERT INTO @DBCC_DBINFO EXECUTE ("DBCC DBINFO ([?]) WITH TABLERESULTS");
SELECT DISTINCT ''?'', [Value] FROM @DBCC_DBINFO WHERE Field = ''dbi_dbccLastKnownGood'';';